This notebook presents an interaction of Diva tools to read data with the leaflet library using the folium package.

You will need to download the Diva read functions from https://github.com/gher-ulg/Diva_python_tools/tree/ReadPlotModule.

In [4]:
import os
import diva2Dread
import folium
import numpy as np

# Get the data

In [5]:
inputdir = './input/'
datafile = os.path.join(inputdir, 'MLD1.dat')
lon, lat, field = diva2Dread.read_data(datafile)

# Prepare the map

We will center the map on the mean location of the data, hoping that there are not bad data.<br>
For the zoom level, you need some tries to find the right level. It has to be an integer.<br>
We will also add a popup for each data marker, so that one can see the actual measurement at that location.

In [6]:
divamap = folium.Map(location=[lat.mean(), lon.mean()], zoom_start=6)
for x, y, MLD in zip(lon, lat, field):
    folium.CircleMarker([y, x], popup="MLD = {0} m".format(np.round(MLD, 2))).add_to(divamap)
divamap